In [1]:
#test1
import tensorflow

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
print('x_train:' , x_train.shape)
K = len(np.unique(y_train)) # Classes
Ntr = x_train.shape[0]

Nte = x_test.shape[0]
Din = 3072 # CIFAR10
# Din = 784 # MINIST
# Normalize pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0
mean_image = np.mean(x_train, axis=0)
x_train = x_train - mean_image
#https://stats.stackexchange.com/questions/211436/why-normalize-images-by-subtracting-datasets-image-mean-instead-of-the-current
x_test = x_test - mean_image
"""In the ytrain it is preliminarily is in numbers just to indicate the particular class tf.keras.utils.to_categorical(y_train, num_classes=K) does is 
Converting the number to a binary vector
Ex: 
a = tf.keras.utils.to_categorical([0, 1, 2, 3], num_classes=4)
a = tf.constant(a, shape=[4, 4])
print(a)
tf.Tensor(
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]], shape=(4, 4), dtype=float32)
"""
y_train = tf.keras.utils.to_categorical(y_train, num_classes=K)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=K)
x_train = np.reshape(x_train,(Ntr,Din))#shaping images into a vector Din=32x32x3
x_test = np.reshape(x_test,(Nte,Din))
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
std=1e-5
w1 = std*np.random.randn(Din, K)#initialization
b1 = np.zeros(K)
print("w1:", w1.shape)
print("b1:", b1.shape)
batch_size = Ntr #for the batch size whole dataset is considered

iterations =300
lr =0.8
lr_decay=0.001#learning rate is decreased in each iteration
reg =0.5#regularization term which stops growth of weights
loss_history = []
train_acc_history = []
val_acc_history = []
"""
seed = 0
rng = np.random.default_rng(seed=seed)
for t in range(iterations):
    indices = np.arange(Ntr)#generating image
    rng.shuffle(indices)
    # Forward pass
    # Backward pass
# Printing accuracies and displaying w as images
"""

x_train: (50000, 32, 32, 3)
w1: (3072, 10)
b1: (10,)


'\nseed = 0\nrng = np.random.default_rng(seed=seed)\nfor t in range(iterations):\n    indices = np.arange(Ntr)#generating image\n    rng.shuffle(indices)\n    # Forward pass\n    # Backward pass\n# Printing accuracies and displaying w as images\n'